In [1]:
import numpy as np
import tensorflow as tf
import pickle
import random
import string
from gui import parse_xml_intents
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KASHISH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KASHISH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\KASHISH\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2025-09-05 13:47:06.181 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 13:47:06.183 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


⚠️  Skipping data\MedQuAD-master\6_NINDS_QA\0000007.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\6_NINDS_QA\0000018.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\6_NINDS_QA\0000182.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\6_NINDS_QA\0000244.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\9_CDC_QA\0000199.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\9_CDC_QA\0000423.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\9_CDC_QA\0000424.xml: <Focus> tag missing or empty.
Parsed 5483 intents from XML


2025-09-05 13:47:07.255 
  command:

    streamlit run c:\Users\KASHISH\Desktop\nullclass_projects\task_2_medical_chatbot\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-05 13:47:07.256 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 13:47:07.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 13:47:07.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 13:47:07.258 Session state does not function when running a script without `streamlit run`
2025-09-05 13:47:07.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 13:47:07.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 13:47:07.261 Thread 'MainThread': missing ScriptRunContext! This warning can b

In [2]:
intent = parse_xml_intents()
if not intent:
    print("No intents found. Please check your XML file.")

⚠️  Skipping data\MedQuAD-master\6_NINDS_QA\0000007.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\6_NINDS_QA\0000018.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\6_NINDS_QA\0000182.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\6_NINDS_QA\0000244.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\9_CDC_QA\0000199.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\9_CDC_QA\0000423.xml: <Focus> tag missing or empty.
⚠️  Skipping data\MedQuAD-master\9_CDC_QA\0000424.xml: <Focus> tag missing or empty.
Parsed 5483 intents from XML


In [3]:
words = []
classes = []
documents = []
ignoreLetters = ["?", "!", ".", ","]

    
for intent in intent:
    for pattern in intent['patterns']:
        wordsList = nltk.word_tokenize(pattern)
        wordsList = [lemmatizer.lemmatize(word.lower()) for word in wordsList if word not in string.punctuation]
        words.extend(wordsList)
        documents.append((wordsList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(set(words))
classes = sorted(set(classes))
    
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [4]:
training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in document[0]]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

In [5]:
random.shuffle(training)
training = np.array(training)
trainX = training[:, : len(words)]
trainY = training[:, len(words):]


In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, input_shape=(len(trainX[0]),), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Early Stopping to Prevent Overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=10, restore_best_weights=True
)
hist = model.fit(np.array(trainX), np.array(trainY), epochs=500, batch_size=8, verbose=1, 
                 validation_split=0.2, callbacks=[early_stopping])

model.save("chatbot_model.h5", hist)
print("Done")

c:\Users\KASHISH\Desktop\nullclass_projects\task_2_medical_chatbot\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - accuracy: 0.0651 - loss: 7.6450 - val_accuracy: 0.2759 - val_loss: 5.5528
Epoch 2/500
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 48s 30ms/step - accuracy: 0.4137 - loss: 4.2559 - val_accuracy: 0.5872 - val_loss: 3.2011
Epoch 3/500
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.6497 - loss: 2.3820 - val_accuracy: 0.6771 - val_loss: 2.7066
Epoch 4/500
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.7217 - loss: 1.6385 - val_accuracy: 0.7101 - val_loss: 2.5853
Epoch 5/500
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.7663 - loss: 1.2081 - val_accuracy: 0.7360 - val_loss: 2.5341
Epoch 6/500
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.8141 - loss: 0.8880 - val_accuracy: 0.7604 - val_loss: 2.5536
Epoch 7/500
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.8577 - loss: 0.6680 - val_accuracy: 0.7634 - val_loss: 2.5441
Epoch 8/500
1640/1640 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 

Done
